In [2]:
rm(list=ls())

library(data.table)
library(tidyverse)
library(rJava)
library(RNetLogo)
library(lhs)

options(warn = -1)

In [3]:
nl.model <- "Segregation"
nl.path <- "C:/Program Files/NetLogo 6.0.4/app"
file.path <- "C:/Users/paslanpatir/Desktop/TEZ_v2/"
model.path <- paste0(file.path,nl.model,".nlogo")

data.path <- "C:/Users/paslanpatir/Desktop/TEZ_v2/data/"
logFile = paste0(data.path,"data_log.txt")

NLStart(nl.path, gui = FALSE,nl.jarname='netlogo-6.0.4.jar', nl.obj = nl.model)
NLLoadModel (model.path, nl.obj = nl.model)

In [4]:
model = ifelse(nl.model == "Segregation","basic","dummy")

In [5]:
## Set model parameters
 # Number of replications for each instance
nofrep = 10 
 # order feature names according to their definition order in run_model
if(nl.model == "Segregation_Dummy"){
    feature_names = c("density","%-similar-wanted","budget-multiplier-dummy","density-multiplier-dummy","noise-dummy","tick-limit")  
 }else if(nl.model == "Segregation"){
    feature_names = c("density","%-similar-wanted")   
}
    # 
output_name = c("percent-similar")

 # Number of input parameters of the agent-based model
nofparams = length(feature_names) 

In [6]:
unlabeled_ins = c(100,200,300,400,500,600,700,800,900) 
test_ins = c(200,300,400)
train_ins_oneshot = 1
train_ins_Ad = 1 

# Functions

In [7]:
#run_model <- function(feature_names,feature_values){ # both should be in character list format
run_model <- function(feature_values){ # both should be in character list format

    
    k = length(feature_names)    
    for(i in 1:k){
        NLCommand(paste0("set ",feature_names[i]," ",feature_values[i]), nl.obj = nl.model)      
    }
    NLCommand("setup", nl.obj = nl.model)
    NLDoCommand(100, "go", nl.obj = nl.model) 
    result <- NLReport(output_name, nl.obj = nl.model)
    return(result)   
}
#run_replicas <- function(nofrep,feature_names,feature_values) {
run_replicas <- function(nofrep,feature_values) {
    replicas = matrix(NA, ncol = nofrep, nrow = 1) # Save the result of each replication
    for(i in 1:nofrep){
     #   replicas[i]= run_model(feature_names,feature_values)
        replicas[i]= run_model(feature_values)
    }
    aggregated_result = mean(replicas)
    return(aggregated_result)
}

#run_ABM = function(nofrep,nofinstances,unlabeledset,featurenames = feature_names){
run_ABM = function(nofrep,nofinstances,unlabeledset){
   #unlabeledset = setcolorder(unlabeledset,featurenames) 
   unlabeledset = setcolorder(unlabeledset,feature_names) 
   for(i in 1:nofinstances){
        #unlabeledset[i, output :=  run_replicas(nofrep,featurenames, as.matrix(unlabeledset[i,]))]    
        unlabeledset[i, output :=  run_replicas(nofrep, as.matrix(unlabeledset[i,]))] 
    } 
    return(unlabeledset)
}


# Unlabeled Pool

In [8]:
for(n in unlabeled_ins){
set.seed(n)
    
unlabeled_pool = as.data.table(maximinLHS(n = n, k = nofparams, dup = 5))

if(model == "basic"){
    unlabeled_pool$V1 = qunif(unlabeled_pool$V1, 10, 90) 
    unlabeled_pool$V2 = qunif(unlabeled_pool$V2, 10, 90)   
 }else if(model != "basic"){   
    unlabeled_pool$V1 = qunif(unlabeled_pool$V1, 10, 90) 
    unlabeled_pool$V2 = qunif(unlabeled_pool$V2, 10, 90)
    unlabeled_pool$V3 = qunif(unlabeled_pool$V3, 1, 10) 
    unlabeled_pool$V4 = qunif(unlabeled_pool$V4, 0.01, 1)    
    unlabeled_pool$V5 = qunif(unlabeled_pool$V5, 0.00001, 0.0001) 
    unlabeled_pool$V6 = qunif(unlabeled_pool$V6, 90, 110)  
}        
    setnames(unlabeled_pool, c(paste0("V",1:nofparams)), feature_names)    
    unlabeled_pool[,idx := 1:.N]
        
    fwrite(unlabeled_pool, paste0(data.path,"unlabeled_pool","_",model,"_",n,"_",Sys.Date(),".csv"))
}

# Test Data

In [ ]:
for(k in test_ins){
set.seed(k)
    
test_set = matrix(nrow = k, ncol = nofparams) %>% data.table()
setnames(test_set, c(paste0("V",1:nofparams)), feature_names)

if(model == "basic"){
    test_set$density            = runif(k, 10, 90) 
    test_set$`%-similar-wanted` = runif(k, 10, 90) 
}else if(model != "basic"){
    test_set$density                    = runif(k, 10, 90) 
    test_set$`%-similar-wanted`         = runif(k, 10, 90) 
    test_set$`budget-multiplier-dummy`  = runif(k, 1, 10) 
    test_set$`density-multiplier-dummy` = runif(k, 0.01, 1) 
    test_set$`noise-dummy`              = runif(k, 0.00001, 0.0001) 
    test_set$`tick-limit`               = runif(k, 90, 110) 
}
 

    test_start = paste0("ABM run start time : ",Sys.time())
    print(test_start)
    write(paste0("test_set","_",model,"_",k,"   ",test_start),logFile, append=TRUE, sep = "\n" )    

    test_set = run_ABM(nofrep,k,test_set) %>% as.data.table()

    test_end = paste0("ABM run end time : ",Sys.time())
    print(test_end)
    write(paste0("test_set","_",model,"_",k,"   ",test_end),logFile, append=TRUE, sep = "\n" )    

fwrite(test_set, paste0(data.path,"test_set","_",model,"_",k,"_",Sys.Date(),".csv"))
}

# Train Data

## One-Shot

In [ ]:
for(m in train_ins_oneshot){
set.seed(m)
    
training_set = as.data.table(maximinLHS(n = m, k = nofparams, dup = 5))

if(model == "basic"){
    training_set$V1 = qunif(training_set$V1, 10, 90) 
    training_set$V2 = qunif(training_set$V2, 10, 90) 
}else if(model != "basic"){  
    training_set$V1 = qunif(training_set$V1, 10, 90) 
    training_set$V2 = qunif(training_set$V2, 10, 90) 
    training_set$V3 = qunif(training_set$V3, 1, 10) 
    training_set$V4 = qunif(training_set$V4, 0.01, 1)    
    training_set$V5 = qunif(training_set$V5, 0.00001, 0.0001) 
    training_set$V6 = qunif(training_set$V6, 90, 110) 
}
    setnames(training_set, c(paste0("V",1:nofparams)), feature_names)
    training_set$output <- 0.00

    training_start = paste0("ABM run start time : ",Sys.time())
    print(training_start)
    write(paste0("training_set","_",model,"_",m,"   ",training_start),logFile, append=TRUE, sep = "\n" )    

    training_set = run_ABM(nofrep,m,training_set) %>% as.data.table()

    training_end = paste0("ABM run end time : ",Sys.time())
    print(training_end)
    write(paste0("training_set","_",model,"_",m,"   ",training_end),logFile, append=TRUE, sep = "\n" )    

fwrite(test_set, paste0(data.path,"training_set","_",model,"_",m,"_",Sys.Date(),".csv"))
    }

## Sequential Samp.

In [ ]:
for(p in train_ins_Ad){
set.seed(p)
    
training_set_Ad = as.data.table(maximinLHS(n = p, k = nofparams, dup = 5))

if(model == "basic"){
    training_set_Ad$V1 = qunif(training_set_Ad$V1, 10, 90) 
    training_set_Ad$V2 = qunif(training_set_Ad$V2, 10, 90)  
}else if(model != "basic"){  
    training_set_Ad$V1 = qunif(training_set_Ad$V1, 10, 90) 
    training_set_Ad$V2 = qunif(training_set_Ad$V2, 10, 90)
    training_set_Ad$V3 = qunif(training_set_Ad$V3, 1, 10) 
    training_set_Ad$V4 = qunif(training_set_Ad$V4, 0.01, 1)    
    training_set_Ad$V5 = qunif(training_set_Ad$V5, 0.00001, 0.0001) 
    training_set_Ad$V6 = qunif(training_set_Ad$V6, 90, 110) 
    
}
    setnames(training_set_Ad, c(paste0("V",1:nofparams)), feature_names)
    training_set_Ad$output <- 0.00
  
    training_Ad_start = paste0("ABM run start time : ",Sys.time())
    print(training_Ad_start)
    write(paste0("training_Ad_set","_",model,"_",p,"   ",training_Ad_start),logFile, append=TRUE, sep = "\n" )    

    training_set_Ad = run_ABM(nofrep,p,training_set_Ad) %>% as.data.table()

    training_Ad_end = paste0("ABM run end time : ",Sys.time())
    print(training_Ad_end)
    write(paste0("training_Ad_set","_",model,"_",p,"   ",training_Ad_end),logFile, append=TRUE, sep = "\n" )    

fwrite(training_set_Ad, paste0(data.path,"training_Ad_set","_",model,"_",p,"_",Sys.Date(),".csv"))
}

In [ ]:
NLQuit(all = TRUE)